## Hands-on Tutorial: Quantifying and Reducing Gender Stereotypes in Word Embeddings

Ensuring fairness in algorithmically-driven decision-making is important to avoid inadvertent cases of bias and perpetuation of harmful stereotypes. However, modern natural language processing techniques, which learn model parameters based on data, might rely on implicit biases presented in the data to make undesirable stereotypical associations. Such a danger is facing us with word embedding, a popular framework to represent text data as vectors which has been used in many machine learning and natural language processing tasks. Recent results ([1](https://arxiv.org/abs/1607.06520), [2](https://arxiv.org/abs/1608.07187)) show that even word embeddings trained on Google News articles exhibit female/male gender stereotypes to a disturbing extent. This raises concerns because of their widespread use, as we describe, often tends to amplify these biases. 

In the following, we provide step-by-step instructions to demonstrate and quanitfy the biases in word embedding.



In [9]:
# Setup:
# Clone the code repository from https://github.com/tolga-b/debiaswe.git
# mkdir debiaswe_tutorial
# cd debiaswe_tutorial
# git clone https://github.com/tolga-b/debiaswe.git

# To reduce the time of downloading data, we provide as subset of GoogleNews-vectors in the following location:
# https://drive.google.com/file/d/1NH6jcrg8SXbnhpIXRIXF_-KUE7wGxGaG/view?usp=sharing

# For full embeddings:
# Download embeddings at https://github.com/tolga-b/debiaswe and put them on the following directory
# embeddings/GoogleNews-vectors-negative300-hard-debiased.bin
# embeddings/GoogleNews-vectors-negative300.bin

In [1]:
from __future__ import print_function, division
%matplotlib inline
from matplotlib import pyplot as plt
import json
import random
import numpy as np

import debiaswe as dwe
import debiaswe.we as we
from debiaswe.we import WordEmbedding
from debiaswe.data import load_professions

## Part 1: Gender Bias in Word Embedding


### Step 1: Load data
We first load the word embedding trained on a corpus of Google News texts consisting of 3 million English words and terms. The embedding maps each word into a 300-dimension vector. 

In [2]:
# load google news word2vec
E = WordEmbedding('./embeddings/itwac128.tsv')

*** Reading data from ./embeddings/itwac128.tsv
(1247492, 128)
1247492 words of dimension 128 : ,, di, ., e, ..., minore-genitore/i, Babelzilla, Spaziolaboratorio, eorismo


In [3]:
# load professions
professions = load_professions()
profession_words = [p[0] for p in professions]

Loaded professions
Format:
word,
definitional female -1.0 -> definitional male 1.0
stereotypical female -1.0 -> stereotypical male 1.0


### Step 2: Define gender direction

We define gender direction by the direciton of she - he because they are frequent and do not have fewer alternative word senses (e.g., man can also refer to mankind). In the paper, we discuss alternative approach for defining gender direction (e.g., using PCA).

In [4]:
# compute gender direction
female_words = ["lei", "donna", "madre", "moglie", "sorella", "femmina"]

male_words = ["lui", "uomo", "padre", "marito", "fratello", "maschio"]
definitional = list(zip(male_words, female_words))

gender_direction = we.doPCA(definitional, E).components_[0]

### Step 3: Generating analogies of "Man: x :: Woman : y"

We show that the word embedding model generates gender-streotypical analogy pairs. 
To generate the analogy pairs, we use the analogy score defined in our paper. This score finds word pairs that are well aligned with gender direction as well as within a short distance from each other to preserve topic consistency. 


In [5]:
# analogies gender
a_gender = E.best_analogies_dist_thresh(gender_direction)

for (a,b,c) in a_gender:
    print(a+"-"+b)

Computing neighbors
Mean: 123.37004
Median: 86.0
sorella-fratello
nonna-nonno
compagna-compagno
madre-padre
ragazza-ragazzo
cugina-cugino
sorellina-fratellino
figlia-figlio
amica-amico
cognata-cognato
figliuola-figliuolo
mamma-papà
suora-prete
suocera-suocero
ragazzina-ragazzino
fidanzata-fidanzato
moglie-marito
donna-uomo
bimba-bimbo
maga-mago
vecchietta-vecchietto
diva-divo
principessa-principe
ballerina-ballerino
zia-nipotino
dama-cavaliere
Ragazza-Ragazzo
poetessa-poeta
attrice-attore
cameriera-cameriere
femmina-maschio
gatta-gatto
padrona-padrone
bambina-bambino
dea-dio
scrittrice-romanziere
pittrice-pittore
fanciulla-guerriero
tipa-tizio
lingua-idioma
mammina-figliolo
nuora-genero
squadra-allenatore
stella-astro
simbologia-simbolismo
dicotomia-dualismo
contessa-conte
musa-cantore
barca-gommone
strega-mostro
canzone-brano
metodologia-metodo
Principessa-Principe
cuoca-cuoco
coordinatrice-coordinatore
signora-signore
nazione-popolo
dottoressa-dottor
bionda-giovanotto
vitalità-dinami

### Step 4: Analyzing gender bias in word vectors asscoiated with professions

Next, we show that many occupations are unintendedly associated with either male of female by projecting their word vectors onto the gender dimension. 

The script will output the profession words sorted with respect to the projection score in the direction of gender.

In [6]:
faculties = ["chimica", "giurisprudenza", "matematica", "odontoiatria", "farmacia", "moda", "fisica",
             "ingegneria", "informatica", "letteratura", "biotecnologie", "medicina", "meccanica", "infermieristica", "statistica", "economia"]


In [7]:
# profession analysis gender
sp = sorted([(E.v(w).dot(gender_direction), w) for w in faculties])
sp

[(-0.058879465, 'infermieristica'),
 (-0.025648367, 'informatica'),
 (-0.023781385, 'biotecnologie'),
 (0.018119212, 'ingegneria'),
 (0.018392682, 'odontoiatria'),
 (0.025583658, 'economia'),
 (0.038101293, 'fisica'),
 (0.05916568, 'statistica'),
 (0.088318035, 'matematica'),
 (0.09959996, 'chimica'),
 (0.10399586, 'meccanica'),
 (0.14511514, 'medicina'),
 (0.1737795, 'giurisprudenza'),
 (0.23350039, 'moda'),
 (0.24890257, 'farmacia'),
 (0.2500486, 'letteratura')]

In [8]:
# profession analysis gender
sp = sorted([(E.v(w).dot(v_gender), w) for w in faculties])

sp[0:20], sp[-20:]

NameError: name 'v_gender' is not defined

## Part 2 Racial Bias

### Step 5: Define racial direction
We define racial direction based on the common names in different Demographic groups. 

In [6]:
names = ["Emily", "Aisha", "Anne", "Keisha", "Jill", "Tamika", "Allison", "Lakisha", "Laurie", "Tanisha", "Sarah",
         "Latoya", "Meredith", "Kenya", "Carrie", "Latonya", "Kristen", "Ebony", "Todd", "Rasheed", "Neil", "Tremayne",
         "Geoffrey", "Kareem", "Brett", "Darnell", "Brendan", "Tyrone", "Greg", "Hakim", "Matthew", "Jamal", "Jay",
         "Leroy", "Brad", "Jermaine"]
names_group1 = [names[2 * i] for i in range(len(names) // 2)]
names_group2 = [names[2 * i + 1] for i in range(len(names) // 2)]

In [7]:
# racial direction
vs = [sum(E.v(w) for w in names) for names in (names_group2, names_group1)]
vs = [v / np.linalg.norm(v) for v in vs]

v_racial = vs[1] - vs[0]
v_racial = v_racial / np.linalg.norm(v_racial)

### Step 6: Generating racial biased analogies

Similar to Step 3, we generate analogies that align with the racial dimension. 

In [8]:
# racial analogies
a_racial = E.best_analogies_dist_thresh(v_racial)

for (a,b,c) in a_racial:
    print(a+"-"+b)

Sarah-Keisha
defensemen-cornerbacks
hipster-hip_hop
punter-cornerback
singer_songwriter-rapper
defenseman-defensive_tackle
pole_vault-triple_jump
musicians-artistes
musician-artiste
catcher-wide_receiver
rock_n_roll-reggae
kicker-kick_returner
tavern-barbershop
freestyle_relay-meter_hurdles
lefthander-swingman
bacon-fried_chicken
artists-rappers
equipment-equipments
hockey-basketball
wool-cotton
unassisted_goal-layup
chocolates-sweets
buddy-cousin
priest-preacher
blue-black
medley_relay-meter_dash
quirky-funky
rabbi-imam
grapes-mango
telecommunications-telecommunication
pitchers-defensive_linemen
passages-verses
er-o
acoustic-soulful
punting-punt_returns
thefts-armed_robbery
bar-nightclub
digs-rebounds
Greg-Geoffrey
cellist-saxophonist
smarts-quickness
puck-halfcourt
quarterback-tailback
fox-leopard
pedophiles-rapists
potatoes-flour
en-el
infrastructure-infrastructural
evangelism-gospel
fiance-aunt
pointers-dunks
baseman-defensive_lineman
pedophile-rapist
joked-smiled
beer-soft_drink
g

### Step 7: Analyzing racial bias in word vectors asscoiated with professions

Similar to Step 4, we project occpurations onto the racial dimension.

In [9]:
# profession analysis racial
sp = sorted([(E.v(w).dot(v_racial), w) for w in profession_words])

sp[0:20], sp[-20:]

([(-0.31546241, u'artiste'),
  (-0.27369621, u'shopkeeper'),
  (-0.27285585, u'taxi_driver'),
  (-0.24248751, u'cab_driver'),
  (-0.23096199, u'preacher'),
  (-0.21709056, u'boxer'),
  (-0.20973532, u'laborer'),
  (-0.20361683, u'barber'),
  (-0.1962502, u'cleric'),
  (-0.18273097, u'bodyguard'),
  (-0.18250427, u'gangster'),
  (-0.18162958, u'singer'),
  (-0.16877081, u'maid'),
  (-0.16871038, u'entertainer'),
  (-0.16197535, u'cabbie'),
  (-0.15332885, u'housewife'),
  (-0.14839591, u'civil_servant'),
  (-0.14115772, u'policeman'),
  (-0.13648951, u'minister'),
  (-0.13296556, u'drug_addict')],
 [(0.08779256, u'organist'),
  (0.090074509, u'philanthropist'),
  (0.09135294, u'cinematographer'),
  (0.093180262, u'manager'),
  (0.093583912, u'investment_banker'),
  (0.096878417, u'professor_emeritus'),
  (0.097828984, u'curator'),
  (0.098648623, u'freelance_writer'),
  (0.099171564, u'programmer'),
  (0.10142015, u'screenwriter'),
  (0.10198847, u'author'),
  (0.10438656, u'inventor'),

## Exercise

Repeat Step 2-4 with debiased word embedding. 

You can use debiaswe debias function to do the debiasing with word sets of your choosing

You can leave equalize_pairs and gender_specific_words blank when coming up with your own groups. We give an example for the case of gender below for you to warm up.

In [14]:
from debiaswe.debias import debias

In [8]:
# Lets load some gender related word lists to help us with debiasing
with open('./data/definitional_pairs.json', "r") as f:
    defs = json.load(f)
print("definitional", defs)

with open('./data/equalize_pairs.json', "r") as f:
    equalize_pairs = json.load(f)

with open('./data/gender_specific_seed.json', "r") as f:
    gender_specific_words = json.load(f)
print("gender specific", len(gender_specific_words), gender_specific_words[:10])

definitional [['woman', 'man'], ['girl', 'boy'], ['she', 'he'], ['mother', 'father'], ['daughter', 'son'], ['gal', 'guy'], ['female', 'male'], ['her', 'his'], ['herself', 'himself'], ['Mary', 'John']]
gender specific 218 ['actress', 'actresses', 'aunt', 'aunts', 'bachelor', 'ballerina', 'barbershop', 'baritone', 'beard', 'beards']


In [9]:
debias(E, gender_specific_words, defs, equalize_pairs)

26423 words of dimension 300 : in, for, that, is, ..., Jay, Leroy, Brad, Jermaine
{('testosterone', 'estrogen'), ('SONS', 'DAUGHTERS'), ('Dudes', 'Gals'), ('PROSTATE_CANCER', 'OVARIAN_CANCER'), ('GENTLEMEN', 'LADIES'), ('gelding', 'mare'), ('men', 'women'), ('brother', 'sister'), ('he', 'she'), ('Brother', 'Sister'), ('KINGS', 'QUEENS'), ('grandson', 'granddaughter'), ('son', 'daughter'), ('Ex_Girlfriend', 'Ex_Boyfriend'), ('His', 'Her'), ('male', 'female'), ('GRANDPA', 'GRANDMA'), ('Males', 'Females'), ('HIS', 'HER'), ('dads', 'moms'), ('PRINCE', 'PRINCESS'), ('SPOKESMAN', 'SPOKESWOMAN'), ('BOY', 'GIRL'), ('Fathers', 'Mothers'), ('his', 'her'), ('Prince', 'Princess'), ('Fella', 'Granny'), ('CATHOLIC_PRIEST', 'NUN'), ('boy', 'girl'), ('WIVES', 'HUSBANDS'), ('councilman', 'councilwoman'), ('prostate_cancer', 'ovarian_cancer'), ('DAD', 'MOM'), ('Grandsons', 'Granddaughters'), ('CHAIRMAN', 'CHAIRWOMAN'), ('Grandfather', 'Grandmother'), ('Boy', 'Girl'), ('Wives', 'Husbands'), ('HIMSELF', '

In [10]:
sp_debiased = sorted([(E.v(w).dot(v_gender), w) for w in faculties])
sp_debiased

[(-0.027093604, 'biotechnology'),
 (-0.026070345, 'engineering'),
 (-0.017902888, 'medicine'),
 (-0.014475763, 'law'),
 (-0.013107251, 'political'),
 (-0.005910009, 'mathematics'),
 (-0.005097652, 'humanities'),
 (-0.0012646113, 'education'),
 (0.0013937466, 'chemistry'),
 (0.019216092, 'physics'),
 (0.025319228, 'pharmacy')]

In [18]:
# profession analysis gender
sp_debiased = sorted([(E.v(w).dot(v_gender), w) for w in profession_words])

sp_debiased[0:20], sp_debiased[-20:]

([(-0.4196325, 'congressman'),
  (-0.40675846, 'businessman'),
  (-0.32398778, 'councilman'),
  (-0.30967093, 'dad'),
  (-0.21665451, 'statesman'),
  (-0.11345412, 'salesman'),
  (-0.07300486, 'monk'),
  (-0.072163954, 'handyman'),
  (-0.04946825, 'minister'),
  (-0.043583885, 'archbishop'),
  (-0.040207203, 'bishop'),
  (-0.038332462, 'commissioner'),
  (-0.035724398, 'surgeon'),
  (-0.03313398, 'trader'),
  (-0.032377202, 'observer'),
  (-0.032095842, 'neurosurgeon'),
  (-0.03145011, 'priest'),
  (-0.031133963, 'skipper'),
  (-0.029659165, 'lawmaker'),
  (-0.029511197, 'commander')],
 [(0.029965654, 'teenager'),
  (0.030237045, 'instructor'),
  (0.030946147, 'student'),
  (0.03111697, 'paralegal'),
  (0.032039426, 'bookkeeper'),
  (0.03243462, 'cinematographer'),
  (0.034329068, 'graphic_designer'),
  (0.03470566, 'lifeguard'),
  (0.035666533, 'janitor'),
  (0.035971954, 'drummer'),
  (0.04212016, 'wrestler'),
  (0.04390225, 'hairdresser'),
  (0.04813315, 'firefighter'),
  (0.2377612

In [11]:
# analogies gender
a_gender_debiased = E.best_analogies_dist_thresh(v_gender)

for (a,b,c) in a_gender_debiased:
    print(a+"-"+b)

Computing neighbors
Mean: 10.218597434053665
Median: 7.0
females-males
ovarian_cancer-prostate_cancer
estrogen-testosterone
grandmother-grandfather
girl-boy
moms-dads
councilwoman-councilman
daughter-son
filly-colt
chairwoman-chairman
gals-dudes
she-he
women-men
girls-boys
granddaughters-grandsons
mare-gelding
queen-king
mothers-fathers
princess-prince
convent-monastery
sorority-fraternity
grandma-grandpa
husbands-wives
twin_sister-twin_brother
sister-brother
sisters-brothers
herself-himself
her-his
female-male
congresswoman-congressman
schoolgirl-schoolboy
businesswoman-businessman
motherhood-fatherhood
ex_boyfriend-ex_girlfriend
aunt-uncle
mother-father
niece-nephew
mom-dad
granddaughter-grandson
woman-man
spokeswoman-spokesman
queens-kings
ladies-gentlemen
daughters-sons
actress-actor
lesbian-gay
compatriot-countryman
husband-younger_brother
gal-dude
hers-theirs
heroine-protagonist
feminism-feminist
actresses-actors
childhood-boyhood
waitress-waiter
kid-guy
me-him
mommy-daddy
aunts-